This will be my notebook for conducting Bayesian mixed-effects logistic regression for power.

I made separate notebooks for this bc it takes forever.

In [1]:
import arviz as az
import bambi as bmb
import argparse
from os.path import join
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from Bayesian_model_utils import run_model, plot_res, print_latex_table, plot_predictions

Once I am running models I can do model comparison using: 

https://arviz-devs.github.io/arviz/api/generated/arviz.compare.html
and 
https://arviz-devs.github.io/arviz/api/generated/arviz.plot_compare.html

as in: https://bambinos.github.io/bambi/main/notebooks/model_comparison.html?highlight=waic


In [2]:
BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subj'}, inplace=True)
BeckScores = BeckScores.drop(columns=['Unnamed: 0'])

pow_df_depr = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)


pow_df_depr = pow_df_depr.drop(columns=['Unnamed: 0'])
pow_df_depr = pow_df_depr.merge(BeckScores, on='subj')


# rename the electrodes to properly do the elec random effects 
pow_df_depr.elec = pow_df_depr.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# Bin the BDI scores (this is ok because of clinical classification). 
pow_df_depr['CBDI'] = pd.qcut(pow_df_depr['BDI'], q=3,
                                                    labels=['low', 'med', 'high'])

# Should I get rid of the neutral CBDI? Not in the original analysis.... 
# pow_df_depr = pow_df_depr[pow_df_depr['CBDI']!='med']

pow_df_depr['valence'] = pow_df_depr['valence'] - pow_df_depr['valence'].mean()

In [ ]:
# Model 6: Effect of depression and power and arousal on memory and binned valence

BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subj'}, inplace=True)
BeckScores = BeckScores.drop(columns=['Unnamed: 0'])

# pow_df_depr = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/LM_data_128_30_ELEC_RESOLVED.csv')
pow_df_depr = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)
# pow_df_depr = pow_df_depr.drop(columns=['Unnamed: 0'])
pow_df_depr = pow_df_depr.merge(BeckScores, on='subj')


# rename the electrodes to properly do the elec random effects 
pow_df_depr.elec = pow_df_depr.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

pow_df_depr['valence'] = pow_df_depr['valence'] - 0.5

# Set categorical orders 
# cat_type = CategoricalDtype(categories=['pos', 'neutral', 'neg'], ordered=True)
# pow_df_depr.CV = pow_df_depr.CV.astype(cat_type)
cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df_depr.region = pow_df_depr.region.astype(cat_type)
# cat_type = CategoricalDtype(categories=['low', 'high'], ordered=True)
# pow_df_depr.CBDI = pow_df_depr.CBDI.astype(cat_type)
cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df_depr.hemi = pow_df_depr.hemi.astype(cat_type)

for band in ['hfa', 'theta']: # theta 
            
#     model_df = pow_df_depr[pow_df_depr.CBDI==depr]
    model_df = pow_df_depr[pow_df_depr.band==band]

    y = 'memory'
    X = ['arousal', 'valence', 'region', 'hemi', 'power', 'BDI'] 
    
    Intx = ['power:arousal', 
            'power:valence',
            'power:hemi',
            'power:region',
            'power:BDI',
            'power:BDI:arousal',
            'power:BDI:valence']
    
    # This is a smaller subset of patients by far. As such, not a lot of room for useless interactions: 
    #             'power:arousal:valence', 'power:region:hemi'

    rand_effect = ['subj', 'elec']
    categorical = ['hemi', 'region']

    # Drop nan data
    model_df = model_df.dropna(subset=X)
    label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_elec_rand_effect'+ 'continuousBDI' + '_nosoz'

    run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
             chains=4, cores=4, tune=1000, draws=1000, label=label)

            

/home1/salman.qasim/miniconda3/envs/bambi_env/lib/python3.10/site-packages/bambi/models.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[obj_cols] = data[obj_cols].apply(lambda x: x.astype("category"))
Modeling the probability that memory==1
Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 39100
Interrupted at 39,099 [78%]: Average Loss = 1.8707e+05
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [1|elec_offset, 1|elec_sigma, 1|subj_offset, 1|subj_sigma, power:BDI:valence, power:BDI:arousal, power:BDI, power:region, power:hemi, power:valence, power:arousal, BDI, power, hemi, region, valence, arousal, Intercept]


In [ ]:
print('done')

In [ ]:
# Model 6: Effect of depression and power and arousal on memory and linear valence

# BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
# BeckScores.rename(columns={'Subject Code':'subj'}, inplace=True)
# BeckScores = BeckScores.drop(columns=['Unnamed: 0'])
# pow_df_depr = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/LM_data_128_30_noIED.csv')
# pow_df_depr = pow_df_depr.drop(columns=['Unnamed: 0'])
# pow_df_depr = pow_df_depr.merge(BeckScores, on='subj')

# for band in ['gamma']:
#     pow_df_depr = pow_df_depr[pow_df_depr.band==band]
        
#     # Set up squared valence 
#     pow_df_depr['valence'] = pow_df_depr['valence'] - pow_df_depr['valence'].mean()
# #     pow_df_depr['valence_squared'] = pow_df_depr['valence']**2
    
#     # Bin the BDI scores (this is ok because of clinical classification). 
#     pow_df_depr['CBDI'] = pd.qcut(pow_df_depr['BDI'], q=3,
#                                                         labels=['low', 'med', 'high'])
#     cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
#     pow_df_depr.region = pow_df_depr.region.astype(cat_type)
    
#     # Should I get rid of the neutral CBDI? Not in the original analysis.... 
#     pow_df_depr = pow_df_depr[pow_df_depr['CBDI']!='med']


#     cat_type = CategoricalDtype(categories=['low', 'high'], ordered=True)
#     pow_df_depr.CBDI = pow_df_depr.CBDI.astype(cat_type)


#     y = 'memory'
#     X = ['arousal', 'valence', 'region', 'power', 'CBDI'] 
#     Intx = ['power:arousal', 
#             'power:valence',
#             'power:CBDI',
#             'power:region',
#             'power:CBDI:valence',
#             'power:CBDI:arousal',
#             'power:CBDI:region',
#             'power:CBDI:region:arousal',
#             'power:CBDI:region:valence']
    
# #             'power:hemi:region:arousal',
# #             'power:hemi:region:CV'

#     rand_effect = 'subj'
#     categorical = ['CBDI', 'region']

#     # Drop nan data
#     pow_df_depr = pow_df_depr.dropna(subset=X)

#     run_model(pow_df_depr, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
#              chains=4, cores=4, tune=500, draws=1000)

In [13]:
# save out the HDI plot of all the fixed effects
band='gamma'
output_dir = '/home1/salman.qasim/Salman_Project/FR_Emotion/BayesModels'
for region in ['Hipp', 'Amy']: 
    for depr in ['low', 'high']: 
        y = 'memory'
        X = ['arousal', 'CV', 'power'] 
        label = (f"{y}" + "_{}"*len(X)).format(*X) + region + band + f'{depr}_BDI'
        results = az.from_netcdf(f'{output_dir}/{label}_model')
        axes = az.plot_forest(results,
                               kind='ridgeplot',
                               var_names=[f'^{x}' for x in X],
                               filter_vars="regex",
                               colors='lightgray',
                               combined=True,
                               hdi_prob=0.95,
                               figsize=(9, 7))
        plt.vlines(0, plt.ylim()[0], plt.ylim()[1], color = 'black')
        plt.xlim([-0.4, 0.4])
        plt.savefig(join(output_dir, f'{label}_HDIplot.pdf'), dpi=300)
        plt.close()